In [28]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import pandas as pd
import numpy as np
import matplotlib.ticker as ticker
from sklearn import preprocessing
%matplotlib inline

In [29]:
df=pd.read_csv('https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv')
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [30]:
df.shape

(194673, 38)

In [31]:
#Clear excess columns

df=df[['SEVERITYCODE','ADDRTYPE','WEATHER','ROADCOND','LIGHTCOND','INCDTTM']]
df.head()

,SEVERITYCODE,ADDRTYPE,WEATHER,ROADCOND,LIGHTCOND,INCDTTM
0,2,Intersection,Overcast,Wet,Daylight,3/27/2013 2:54:00 PM
1,1,Block,Raining,Wet,Dark - Street Lights On,12/20/2006 6:55:00 PM
2,1,Block,Overcast,Dry,Daylight,11/18/2004 10:20:00 AM
3,1,Block,Clear,Dry,Daylight,3/29/2013 9:26:00 AM
4,2,Intersection,Raining,Wet,Daylight,1/28/2004 8:04:00 AM


In [32]:
#Feature 1

df.groupby(['ADDRTYPE'])['SEVERITYCODE'].value_counts(normalize=True)

ADDRTYPE      SEVERITYCODE
Alley         1               0.890812
              2               0.109188
Block         1               0.762885
              2               0.237115
Intersection  1               0.572476
              2               0.427524
Name: SEVERITYCODE, dtype: float64

In [33]:
#Feature 2

df.groupby(['WEATHER'])['SEVERITYCODE'].value_counts(normalize=True)

WEATHER                   SEVERITYCODE
Blowing Sand/Dirt         1               0.732143
                          2               0.267857
Clear                     1               0.677509
                          2               0.322491
Fog/Smog/Smoke            1               0.671353
                          2               0.328647
Other                     1               0.860577
                          2               0.139423
Overcast                  1               0.684456
                          2               0.315544
Partly Cloudy             2               0.600000
                          1               0.400000
Raining                   1               0.662815
                          2               0.337185
Severe Crosswind          1               0.720000
                          2               0.280000
Sleet/Hail/Freezing Rain  1               0.752212
                          2               0.247788
Snowing                   1               0

In [35]:
#Feature 3

df.groupby(['ROADCOND'])['SEVERITYCODE'].value_counts(normalize=True)

ROADCOND        SEVERITYCODE
Dry             1               0.678227
                2               0.321773
Ice             1               0.774194
                2               0.225806
Oil             1               0.625000
                2               0.375000
Other           1               0.674242
                2               0.325758
Sand/Mud/Dirt   1               0.693333
                2               0.306667
Snow/Slush      1               0.833665
                2               0.166335
Standing Water  1               0.739130
                2               0.260870
Unknown         1               0.950325
                2               0.049675
Wet             1               0.668134
                2               0.331866
Name: SEVERITYCODE, dtype: float64

In [34]:
#Feature 4

df.groupby(['LIGHTCOND'])['SEVERITYCODE'].value_counts(normalize=True)

LIGHTCOND                 SEVERITYCODE
Dark - No Street Lights   1               0.782694
                          2               0.217306
Dark - Street Lights Off  1               0.736447
                          2               0.263553
Dark - Street Lights On   1               0.701589
                          2               0.298411
Dark - Unknown Lighting   1               0.636364
                          2               0.363636
Dawn                      1               0.670663
                          2               0.329337
Daylight                  1               0.668116
                          2               0.331884
Dusk                      1               0.670620
                          2               0.329380
Other                     1               0.778723
                          2               0.221277
Unknown                   1               0.955095
                          2               0.044905
Name: SEVERITYCODE, dtype: float64

In [36]:
#Feature 5
df['INCDTTM'] = pd.to_datetime(df['INCDTTM'])
df['DAYOFWEEK'] = df['INCDTTM'].dt.dayofweek
df.head()

,SEVERITYCODE,ADDRTYPE,WEATHER,ROADCOND,LIGHTCOND,INCDTTM,DAYOFWEEK
0,2,Intersection,Overcast,Wet,Daylight,2013-03-27 14:54:00,2
1,1,Block,Raining,Wet,Dark - Street Lights On,2006-12-20 18:55:00,2
2,1,Block,Overcast,Dry,Daylight,2004-11-18 10:20:00,3
3,1,Block,Clear,Dry,Daylight,2013-03-29 09:26:00,4
4,2,Intersection,Raining,Wet,Daylight,2004-01-28 08:04:00,2


In [37]:
df.groupby(['DAYOFWEEK'])['SEVERITYCODE'].value_counts(normalize=True)

DAYOFWEEK  SEVERITYCODE
0          1               0.697281
           2               0.302719
1          1               0.694250
           2               0.305750
2          1               0.695705
           2               0.304295
3          1               0.692470
           2               0.307530
4          1               0.704358
           2               0.295642
5          1               0.706196
           2               0.293804
6          1               0.722022
           2               0.277978
Name: SEVERITYCODE, dtype: float64

In [38]:
Feature = df[['DAYOFWEEK']]
Feature = pd.concat([Feature,pd.get_dummies(df['ADDRTYPE']),pd.get_dummies(df['WEATHER']),pd.get_dummies(df['ROADCOND']),pd.get_dummies(df['LIGHTCOND'])], axis=1)
Feature.head()

,DAYOFWEEK,Alley,Block,Intersection,Blowing Sand/Dirt,Clear,Fog/Smog/Smoke,Other,Overcast,Partly Cloudy,...,Wet,Dark - No Street Lights,Dark - Street Lights Off,Dark - Street Lights On,Dark - Unknown Lighting,Dawn,Daylight,Dusk,Other,Unknown
0,2,0,0,1,0,0,0,0,1,0,...,1,0,0,0,0,0,1,0,0,0
1,2,0,1,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
2,3,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
3,4,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2,0,0,1,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0


In [39]:
X = Feature
y = df['SEVERITYCODE'].values


In [13]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:1: DataConversionWarning: Data with input dtype uint8, int64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


array([[-0.48852801, -0.06223095, -1.36876932,  1.41129279, -0.01696304,
        -1.15340914, -0.05414257, -0.06551471,  2.45445634, -0.00506801,
        -0.45298634, -0.011333  , -0.02409974, -0.06841713, -0.28988624,
        -1.33213439, -0.07905204, -0.01813462, -0.02604842, -0.01963186,
        -0.07200071, -0.02431221, -0.28975087,  1.76085874, -0.08920831,
        -0.07872239, -0.576075  , -0.00751719, -0.1141037 ,  0.82233559,
        -0.17682024, -0.03476509, -0.27267986],
       [-0.48852801, -0.06223095,  0.73058329, -0.70857019, -0.01696304,
        -1.15340914, -0.05414257, -0.06551471, -0.4074222 , -0.00506801,
         2.2075721 , -0.011333  , -0.02409974, -0.06841713, -0.28988624,
        -1.33213439, -0.07905204, -0.01813462, -0.02604842, -0.01963186,
        -0.07200071, -0.02431221, -0.28975087,  1.76085874, -0.08920831,
        -0.07872239,  1.73588509, -0.00751719, -0.1141037 , -1.21604855,
        -0.17682024, -0.03476509, -0.27267986],
       [ 0.03188685, -0.0622

In [40]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_similarity_score, log_loss, f1_score
from sklearn import metrics

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9)
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (19467, 33) (19467,)
Test set: (175206, 33) (175206,)


In [46]:
#KNN
k = 10
neigh = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)
neigh

KNN = neigh.predict(X_test)
KNN[0:5]

from sklearn import metrics
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, KNN))
print("Train set jaccard similarity score: ", jaccard_similarity_score(y_train, neigh.predict(X_train)))
print("Valid set jaccard similarity score: ", jaccard_similarity_score(y_test, neigh.predict(X_test)))
print("Train set f1 score: ", f1_score(y_train, neigh.predict(X_train), average='weighted'))
print("Valid set f1 score: ", f1_score(y_test, neigh.predict(X_test), average='weighted'))

Train set Accuracy:  0.6932758000719166
Test set Accuracy:  0.6866545666244307
Train set jaccard similarity score:  0.6932758000719166
Valid set jaccard similarity score:  0.6866545666244307
Train set f1 score:  0.6474627015646283
Valid set f1 score:  0.6408401619312328


In [47]:
# Decision Tree

from sklearn.tree import DecisionTreeClassifier
Tree = DecisionTreeClassifier(criterion="entropy")
Tree.fit(X_train,y_train)
predTree = Tree.predict(X_test)
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, KNN))
print("Train set jaccard similarity score: ", jaccard_similarity_score(y_train, neigh.predict(X_train)))
print("Valid set jaccard similarity score: ", jaccard_similarity_score(y_test, neigh.predict(X_test)))
print("Train set f1 score: ", f1_score(y_train, neigh.predict(X_train), average='weighted'))
print("Valid set f1 score: ", f1_score(y_test, neigh.predict(X_test), average='weighted'))

DecisionTrees's Accuracy:  0.6945195940778284
Train set Accuracy:  0.6932758000719166
Test set Accuracy:  0.6866545666244307
Train set jaccard similarity score:  0.6932758000719166
Valid set jaccard similarity score:  0.6866545666244307
Train set f1 score:  0.6474627015646283
Valid set f1 score:  0.6408401619312328


In [48]:
# Support Vector Machine

from sklearn import svm
clf = svm.SVC(kernel='sigmoid')
clf.fit(X_train, y_train)
SVM = clf.predict(X_test)
SVM [0:5]
from sklearn.metrics import f1_score
print("Logistic Regression's Accuracy: ", metrics.accuracy_score(y_test, SVM))
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, KNN))
print("Train set jaccard similarity score: ", jaccard_similarity_score(y_train, neigh.predict(X_train)))
print("Valid set jaccard similarity score: ", jaccard_similarity_score(y_test, neigh.predict(X_test)))
print("Train set f1 score: ", f1_score(y_train, neigh.predict(X_train), average='weighted'))
print("Valid set f1 score: ", f1_score(y_test, neigh.predict(X_test), average='weighted'))

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Logistic Regression's Accuracy:  0.6099049119322397
Train set Accuracy:  0.6932758000719166
Test set Accuracy:  0.6866545666244307
Train set jaccard similarity score:  0.6932758000719166
Valid set jaccard similarity score:  0.6866545666244307
Train set f1 score:  0.6474627015646283
Valid set f1 score:  0.6408401619312328


In [49]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,y_train)
LRyhat = LR.predict(X_test)
print("Logistic Regression's Accuracy: ", metrics.accuracy_score(y_test, LRyhat))
print("Train set Accuracy: ", metrics.accuracy_score(y_train, neigh.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(y_test, neigh.predict(X_test)))
print("Train set jaccard similarity score: ", jaccard_similarity_score(y_train, neigh.predict(X_train)))
print("Valid set jaccard similarity score: ", jaccard_similarity_score(y_test, neigh.predict(X_test)))
print("Train set f1 score: ", f1_score(y_train, neigh.predict(X_train), average='weighted'))
print("Valid set f1 score: ", f1_score(y_test, neigh.predict(X_test), average='weighted'))

Logistic Regression's Accuracy:  0.700991975160668
Train set Accuracy:  0.6932758000719166
Test set Accuracy:  0.6866545666244307
Train set jaccard similarity score:  0.6932758000719166
Valid set jaccard similarity score:  0.6866545666244307
Train set f1 score:  0.6474627015646283
Valid set f1 score:  0.6408401619312328


NameError: name 'yhat_prob' is not defined

In [ ]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

In [ ]:
KNN_pred = neigh.predict(X_test)
DT_pred = loanTree.predict(X_test)
svm_pred = clf.predict(X_test)
LR_pred = LR.predict(X_test)

jaccard_score = [jaccard_similarity_score(y_test, KNN_pred),
                 jaccard_similarity_score(y_test, DT_pred),
                 jaccard_similarity_score(y_test, svm_pred),
                 jaccard_similarity_score(y_test, LR_pred)]


F1_score = [f1_score(y_test, KNN_pred, average='weighted'),
            f1_score(y_test, DT_pred, average='weighted'),
            f1_score(y_test, svm_pred, average='weighted'),
            f1_score(y_test, LR_pred, average='weighted')]

LR_pred_1 = (LR_pred == '2')
y_test_1 = (y_test == '2')
Logloss_score = ['NA','NA','NA',log_loss(y_test_1, LR_pred_1)]


df = {'Algorithm': ['KNN', 'Decistion Tree', 'SVM', 'LogisticRegression'], \
     'Jaccard': jaccard_score, 'F1-score': F1_score, 'LogLoss': Logloss_score}
df_final = pd.DataFrame(data=df, columns=['Algorithm', 'Jaccard', 'F1-score', 'LogLoss'], index=None)
df_final